Mounting my Google Drive in order to access the data

Reading the first line from the website dataset to understand the structure

In [1]:
import pandas as pd
import gcsfs
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [4]:
pathFiles = 'soleadify_sample_data/raw_data/'

In [8]:
with gcs.open(pathFiles+'website dataset.csv') as f:
    line = f.readline()

print(line)

b'root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category,,,,,,\r\n'


Seems like there are several columns at the end that do not contain any sort of information. I will try to read the whole dataset and if these columns do not contain anything I will discard them.

In [9]:
with gcs.open(pathFiles+'website dataset.csv') as f:
    website_dataset = pd.read_csv(f)
website_dataset.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (12,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores,NaN,NaN,NaN,NaN,NaN,NaN
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers,NaN,NaN,NaN,NaN,NaN,NaN
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers,NaN,NaN,NaN,NaN,NaN,NaN
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
website_dataset.columns

Index(['root_domain', 'domain_suffix', 'language', 'legal_name', 'main_city',
       'main_country', 'main_region', 'phone', 'site_name', 'tld',
       's_category', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [ ]:
columns = ['Unnamed: '+str(i) for i in range(11,17)]
print('Website dataset has',website_dataset.shape, 'rows')
extra_columns = website_dataset[columns].apply(lambda col: col.notna()).sum()
print('In the unnamed columns we have the following number of records which are not null')
extra_columns

Website dataset has (71917, 17) rows
In the unnamed columns we have the following number of records which are not null


Unnamed: 11    8
Unnamed: 12    3
Unnamed: 13    2
Unnamed: 14    2
Unnamed: 15    2
Unnamed: 16    2
dtype: int64

Seems like the last 6 columns do not have a name nor do they contain any sort of information. Therefore I will discard them as they do not bring value.

In [10]:
columns_to_include = [x for x in range(11)]
with gcs.open(pathFiles+'website dataset.csv') as f:
    website_dataset = pd.read_csv(f, usecols = columns_to_include)


In [ ]:
website_dataset.head(10)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting
5,ironcrow.ca,ca,en,NaN,calgary,canada,alberta,14032878770,Iron Crow,ca,Furniture Stores
6,springboarddm.com,com,en,The Hershey Company,mississauga,canada,ontario,19053690553,SpringBoard Data Management,com,Data Solutions
7,stoneandtilesrus.com,com,en,NaN,brampton,canada,ontario,19054940660,STONE AND TILES R US,com,Tile Store
8,mdaccpap.com,com,en,MDAC CPAP INC,thunder bay,canada,ontario,18076834405,MDAC CPAP,com,Medical Supply Manufacturers
9,micacchi.ca,ca,en,NaN,toronto,canada,ontario,16477257799,Micacchi Architecture,ca,Architects & Architectural Services


In [11]:
with gcs.open(pathFiles+'google dataset.csv') as f:
    google_dataset = pd.read_csv(f)
google_dataset.head()

ParserError: Error tokenizing data. C error: Expected 15 fields in line 18, saw 16


It seems that Line 18 of the google dataset has more fields than expecting. I will read 1 line from the beginning and line 18 to compare them. Then I will decide on the approach

In [12]:
with gcs.open(pathFiles+'google dataset.csv') as f:
    for index, line in enumerate(f):
        if index == 0:
            header = line
        elif index == 1:
            line_1 = line
        elif index == 18:
            line_18 = line
        elif index>18:
            break
print(header)
print(line_1)
print(line_18)

b'address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain\n'
b'"28 Central Coast Hwy, West Gosford NSW 2250, Australia",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,+61243355946,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) \xc2\xb7 Craft store West Gosford NSW, Australia Closed \xe2\x8b\x85 Opens 9AM Sun \xc2\xb7 +61 2 4335 5946 In-store pickup\xc2\xb7In-store shopping",2250,spotlightstores.com\n'
b'"363 Horton St E, London, ON N6B 1L6, Canada",Beauty Salons - Equipment & Supplies,london,"",canada,Halo Hair Co,"","",363 Horton St E,"",on,ontario,"4.7 (3) \xc2\xb7 Hairdresser 363 Horton St E Closed \xe2\x8b\x85 Opens 10AM Tue \\"Loved the energy and the vibe in the salon.. and love my hair!!!\\"",n6b 1l6,instagram.com\n'


In [13]:
with gcs.open(pathFiles+'google dataset.csv') as f:
    google_dataset = pd.read_csv(f, quotechar='"', escapechar='\\')

In [ ]:
google_dataset.head(10)

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,gosford,au,australia,Spotlight West Gosford,6.124336e+10,au,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,st. catharines,ca,canada,Heritage Christian Book Store,1.905937e+10,ca,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,yarmouth,ca,canada,Pleasant Timber Mart,1.902743e+10,ca,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,los angeles,us,united states,Skin Specifics Medical Spa West LA,1.818427e+10,us,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee
4,NaN,Catering & Delivery,NaN,NaN,NaN,Lakay express,NaN,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee
5,"Ferndale, MI, United States",Pubs & Bars,ferndale,us,united states,B. Nektar Meadery - Taproom & Headquarters,1.313745e+10,us,"Ferndale, MI, United States",+1 313-744-6323,mi,michigan,"4.8 (296) · $$ · Bar Ferndale, MI, United Stat...",NaN,linktr.ee
6,"321 Kent St, Sydney NSW 2000, Australia","Insurance - Agents, Carriers & Brokers",sydney,au,australia,icare NSW,NaN,NaN,"Sydney NSW, Australia",NaN,nsw,new south wales,"2.3 (187) · Insurance company Sydney NSW, Aust...",2000,icare.nsw.gov.au
7,"55 S Cleveland Ave, Westerville, OH 43081",Preschools & Kindergartens,westerville,us,united states,South Cleveland Avenue KinderCare,1.614899e+10,NaN,"7+ years in business · Westerville, OH",(614) 899-0026,oh,ohio,3.6 (20) · Preschool 7+ years in business · We...,43081,kindercare.com
8,"4050 S Torrey Pines Dr, Las Vegas, NV 89103, U...",Preschools & Kindergartens,las vegas,us,united states,Torrey Pines KinderCare,1.702367e+10,us,"7+ years in business · Las Vegas, NV, United S...",+1 702-367-0822,nv,nevada,3.8 (11) · Preschool 7+ years in business · La...,89103,kindercare.com
9,"7+ years in business · St Paul, MN, United States",Preschools & Kindergartens,NaN,us,NaN,Meadowlands KinderCare,1.651430e+10,us,"7+ years in business · St Paul, MN, United States",(651) 429-6316,NaN,NaN,4.8 (9) · Kindergarten 1000 Meadowlands Dr Clo...,NaN,kindercare.com


Seems like adding an escape char worked, google data has been read.

In [14]:
with gcs.open(pathFiles+'facebook dataset.csv') as f:
    facebook_dataset = pd.read_csv(f, quotechar='"', escapechar='\\')

In [ ]:
facebook_dataset.head(10)

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,NaN,occ,occitanie,30420
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,ca,NaN,NaN,NaN
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,ca,bc,british columbia,v6b 1g1
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,ca,mb,manitoba,r3l 2t7
5,collins-cc.edu,NaN,Other schools|High Schools|Community Center|Ad...,NaN,NaN,NaN,NaN,NaN,https://collins-cc.edu,Collins Career Technical Center,LocalBusiness,NaN,NaN,NaN,NaN,NaN
6,dovercheese.com,"307 main street, n0a 1n0, port dover, on, cana...",NaN,port dover,ca,canada,The Dover Cheese Shop is a food lover’s paradi...,info@dovercheese.com,http://dovercheese.com,The Dover Cheese Shop,LocalBusiness,NaN,NaN,on,ontario,n0a 1n0
7,alphascientific.ca,"8623 granville street unit 143, v6p 5a2, vanco...",Medical Supply Manufacturers,vancouver,ca,canada,www.alphascientific.ca Alpha Scientific Repair...,sales@alphascientific.ca,http://alphascientific.ca,Alpha Scientific Repair Services Ltd.,LocalBusiness,1.604690e+10,ca,bc,british columbia,v6p 5a2
8,modernmama.com,NaN,Book Publisher,NaN,NaN,NaN,NaN,NaN,https://modernmama.com/edmonton,Modern Mama Edmonton,Person,NaN,NaN,NaN,NaN,NaN
9,unitysodfarm.com,"2989 unity road, k0h 1m0, kingston, on, canada...",Agriculture & Farm Associations|Garden Equipme...,NaN,ca,NaN,"Kingston's local supplier of quality sod, seed...",info@unitysodfarm.com,http://unitysodfarm.com,Unity Sod Farm Ltd,LocalBusiness,1.613389e+10,ca,NaN,NaN,NaN


I will next evaluate the information available in the 3 datasets:
*   what columns are in each dataset?
    - which are common? -> could be used as linkage keys
    - which are unique -> useful to expand the information available
*   how many companies are in each dataset?
    - if there is no overlap then in total I should have the sum of rows of the 3 datasets
    - there should be at least some form of overlapping -> how do I evaluate how much data I have to discard due to linkaged mismatch?



In [15]:
print('Website columns', website_dataset.columns)
print('Google columns', google_dataset.columns)
print('Facebook columns', facebook_dataset.columns)

Website columns Index(['root_domain', 'domain_suffix', 'language', 'legal_name', 'main_city',
       'main_country', 'main_region', 'phone', 'site_name', 'tld',
       's_category'],
      dtype='object')
Google columns Index(['address', 'category', 'city', 'country_code', 'country_name', 'name',
       'phone', 'phone_country_code', 'raw_address', 'raw_phone',
       'region_code', 'region_name', 'text', 'zip_code', 'domain'],
      dtype='object')
Facebook columns Index(['domain', 'address', 'categories', 'city', 'country_code',
       'country_name', 'description', 'email', 'link', 'name', 'page_type',
       'phone', 'phone_country_code', 'region_code', 'region_name',
       'zip_code'],
      dtype='object')


It looks that there are a lot of columns that are common. However it seems that the same column is slightly different in each dataset.

I will divide the columns into several clusters:
Cluster 1 - Columns which can be used to split the datasets into smaller chuks which can be easier to link. For example it should be easier to link Insurance companies from USA, New York across the 3 datasets, than linking everything using all columns. The matching algorithm will have to work on a smaller set -> probably better results and performance.

Here I include geographical information such as
*   City
    - Websiste -> called main_city
    - Google -> called city
    - Facebook -> called city
*   Country
    - Website -> called main_country
    - Google -> called country_name
    - Facebook -> called country_name
*    Region
    - Website -> called main_region
    - Google -> called region_name; also available region_code
    - Facebook -> called region_name; also available region_code

On top of that between Google and Facebook datasets there seems to be also the Zip Code which is common and could be used to further narrow down the matching. Between these 2 it seems that we also have some form of address field in both datasets. Could be useful as well.

Also the category could be useful. Howver across the 3 datasets this field seems to store slightly different information.
*   Category
    - Website -> called s_category -> this seems to include only top level information such as "Business Consulting", "Furniture Store"
    - Google -> called category -> this seems to include only top level information such as "Book Stores", "Catering & Delivery"
    - Facebook -> called categories. It seems to be a much more detailed Category Tree such as 

Cluster 2 - Columns which can be used to uniquely identify a company. They could be used individually or combined.


*   Domain
    - Website -> called root_domain
    - Google -> called domain
    - Facebook -> called domain
*   Company name
    - Website -> 2 fields: site_name, legal_name
    - Google -> called name
    - Facebook -> called name
*   Phone
    - Websiste -> called phone. It seems to be a number stored as string, however it does not include the + as part of the dialing code.
    - Google -> 2 fields: phone, string as number, raw_phone, string which includes the dialing code
    - Facebook -> phone, string as number

Between Google and Facebook there is also the phone_country_code which is common and could be used together with the Geographical information to match the data. Besides this, the address field is also common. However it seems that the address field is slightly different. It will need more investigation before using it

Finally there is a set of information which is unique to a dataset:


*   Website: legal_name, language and tld (top-level domain). Out of these 3, the legal_name is rather unique and possibly expanding the information available. Language and tld don't seem to be that useful. 
*   Google: text. It seems to be a combination of address, Google Rating and something else.
*   Facebook: description, email, link, page_type. Out of these, the link is the least useful information. It is to some extent already included in the domain. Protocol is however not included here. Email, description are extremly useful and unique. Page Type can also be useful, but it's mainly subject to the definition used by Facebook.








In [ ]:
print('Website dataset has {0:,} rows'.format(website_dataset.shape[0]))
print('Google dataset has {0:,} rows'.format(google_dataset.shape[0]))
print('Facebook dataset has {0:,} rows'.format(facebook_dataset.shape[0]))

Website dataset has 71,917 rows
Google dataset has 356,520 rows
Facebook dataset has 72,010 rows


Based on the analysis of common columns and number of rows, the high level strategy will be to
Step 1: Link Google and Facebook data into a single dataset
Step 2: Link the combined dataset with the Website dataset

Reason for this approach is:
*   Google and Facebook have a few more columns in common. To be investigated to what extent this columns actually have data and how information is structured in them.
*   Google is the dataset with the most rows. Standalone it has the most value. The other 2 datasets have only 20% of the number of rows. Therefore at best they could: either increase the dataset with 20% more rows (union - no common rows) or increase the information available for 20% of the Google dataset (join - all rows are in common).

Next step is to explore the quality of each dataset:
*    How many missing values in each column?
*    Are the countries / regions consistent or are there variations - UPPERCASE, lowercase, Titlecase etc.
*    Are there duplicates?

In [ ]:
website_dataset.describe()

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category
count,71917,71803,66753,31926,60924,64835,60900,65618,68695,71803,70677
unique,71917,605,13,30586,9576,253,1067,61921,67529,301,568
top,bbqgourmet.com,com,en,"Blackboard, Inc.",toronto,canada,ontario,6.11301E+11,Mysite,com,"Executive, Legislative & Government Institutions"
freq,1,33532,56181,211,3293,37628,20032,184,40,33532,1508


Based on the output above:
*    root_domain can be used to uniquely identify a row. Count is equal to Unique therefore no duplicates.
*    legal_name - count is higher than unique. This means that some root_domains are associated to the same legal_names. There are duplicates in this column
*    site_name seems to cover more of the data than legal_name. However the degree of duplication is higher.
*    unique country count is 253. There are only 195 countries in the world -> there is duplication also in the form of inconsistent writing of the same information.
*    phone - unique is lower than count. Again a case of duplication

In [ ]:
# I want to check that the root_domain is truly unique or actually duplication exists, it's just hidden by Case variation
(website_dataset['root_domain'].apply(lambda x: (x.lower()).strip())).describe()
# It seems that is indeed unique, that is good

count              71917
unique             71917
top       bbqgourmet.com
freq                   1
Name: root_domain, dtype: object

In [16]:
site_names_raw = website_dataset.groupby('site_name')['root_domain'].count().sort_values(ascending = False)
print(site_names_raw.head(30))
# Based on this output I'm going to apply some cleaning on this column
import numpy as np
replacements = ['Mysite','mysite', 'My Site', 'My site','Home', 'website', '#NAME?']
website_dataset['site_name_clean']=website_dataset['site_name'].replace(replacements, [np.nan] * len(replacements)).str.strip()

site_names_cleaned = website_dataset.groupby('site_name_clean')['root_domain'].count().sort_values(ascending = False)
print(site_names_cleaned.head(30))
print(site_names_cleaned.tail(30))

site_name
Mysite                                  40
mysite                                  29
The Canadian Real Estate                27
Home                                    24
InMotion Hosting                        16
Desjardins Insurance                    15
InterWorx                               15
website                                 14
#NAME?                                  10
SchoolMessenger                         10
ShopCity.com                             8
Hosting                                  8
TCS                                      8
Royal LePage                             8
Website.com Solutions                    8
Net Driven                               8
NodeWorx                                 8
CFA                                      7
Red Deer                                 7
AMVOQ                                    7
Standard Auto Glass                      7
Garaga                                   7
Directwest                               6
U

In [17]:
categories_raw = website_dataset.groupby('s_category')['root_domain'].count().sort_values(ascending = False)
print(categories_raw.head(30))
print(categories_raw.tail(30))

# Based on the output there are a few instances that can be replaced and also some case incosistency
replacements = ['com','org', 'ru']
website_dataset['category_clean']=(website_dataset['s_category'].replace(replacements, [np.nan] * len(replacements)).str.strip()).str.title()

categories_cleaned = website_dataset.groupby('category_clean')['root_domain'].count().sort_values(ascending = False)
print(categories_cleaned.head(30))
print(categories_cleaned.tail(30))

s_category
Executive, Legislative & Government Institutions    1508
Colleges & Universities                             1323
High Schools                                        1277
Home Builders & Renovation Contractors               919
Automobile Dealers & Manufacturers                   873
Parked Domains                                       785
Auto Services                                        756
Tech Companies                                       753
Real Estate - Agents & Managers                      738
Bakeries & Desserts                                  686
Commercial Printing                                  674
Landscaping & Lawn Services                          653
Churches & Religious Organizations                   632
Yoga Studios                                         629
Heating & Air Conditioning                           601
Oil & Gas - Extraction & Distribution                561
Social Services & NGOs                               508
Employment Agencies 

In [20]:
legal_names_raw = website_dataset.groupby('legal_name')['root_domain'].count().sort_values(ascending = False)
print(legal_names_raw.head(60))
print(legal_names_raw.tail(60))

# Several action points - strings Inc. INC represent the same thing, company type
# -> All uppercase could solve the problem combined with a removal of the . at the end
# Strings- , Inc is the same as Inc
# -> Replacing the , with an empty characther should solve it
# Ltd and Limited are the same thing. Inc and Incorporated are the same thing
# Corporation, Corp are the same thing

def clean_legal_name(legal_name):
    if not pd.isna(legal_name):
        out = legal_name.rstrip(' .').replace(',','').upper()
        if out.find('LIMITED')>-1:
            out = out.replace('LIMITED','LTD')
        elif out.find('INCORPORATED')>-1:
            out = out.replace('INCORPORATED','INC')
        elif out.find('CORPORATION')>-1:
            out = out.replace('CORPORATION','CORP')
        else:
            out = legal_name
        
        return out
    else:
        return np.nan


website_dataset['legal_name_clean'] = website_dataset['legal_name'].apply(clean_legal_name)

legal_names_cleaned = website_dataset.groupby('legal_name_clean')['root_domain'].count().sort_values(ascending = False)
print(legal_names_cleaned.head(60))
print(legal_names_cleaned.tail(60))

legal_name
Blackboard, Inc.                                  211
LifeLearn Animal Health (LifeLearn Inc.)           52
Dealer Spike, LLC                                  50
REALTOR(r) Canada Inc.                             31
Desjardins Financial Security Investments Inc.     27
TouchBistro Inc.                                   27
Google, Inc.                                       26
FCA Canada Inc.                                    22
Google Inc.                                        21
PowerSchool Group LLC                              21
MH Sub I, LLC                                      21
General Motors of Canada Company                   17
Harvest Digital Planning Pty Ltd                   16
Lovingly, LLC                                      13
Innovative Interfaces, Inc.                        12
Dominion Lending Centres Inc.                      11
Apple Inc.                                         11
Google LLC                                         11
Facebook Inc.    

In [21]:
countries_raw = website_dataset.groupby('main_country')['root_domain'].count().sort_values(ascending = False)
print(countries_raw.head(60))
print(countries_raw.tail(60))

#incosistent casing - Greece and greece, Taiwan and taiwan are the same thing

def clean_country(country):
    if not pd.isna(country):
        if country == ')"':
            return np.nan
        else:
            return country.strip().title()
    else:
        return np.nan

website_dataset['country_clean'] = website_dataset['main_country'].apply(clean_country)

countries_cleaned = website_dataset.groupby('country_clean')['root_domain'].count().sort_values(ascending = False)
print(countries_cleaned.head(60))
print(countries_cleaned.tail(60))

main_country
canada                  37628
united states            9503
australia                2482
Canada                   2151
united kingdom           1820
turkey                    911
russia                    698
netherlands               624
france                    609
germany                   555
south africa              539
japan                     506
united arab emirates      435
malaysia                  428
india                     405
belgium                   377
china                     255
singapore                 232
new zealand               225
hong kong                 199
Australia                 193
spain                     177
poland                    173
greece                    162
Greece                    159
italy                     153
Taiwan                    141
taiwan                    135
switzerland               110
qatar                     106
Turkey                     94
hungary                    94
philippines                

In [23]:
regions_raw = website_dataset.groupby('main_region')['root_domain'].count().sort_values(ascending = False)
print(regions_raw.head(60))
print(regions_raw.tail(60))

def clean_region(region):
    if not pd.isna(region):
        return region.strip(" '").strip('"').replace("'"," ").title()
    else:
        return np.nan

website_dataset['region_clean']=website_dataset['main_region'].apply(clean_region)

main_region
ontario                   20032
alberta                    7325
quebec                     4816
british columbia           2197
england                    1711
saskatchewan               1419
manitoba                   1195
texas                      1043
california                  859
victoria                    797
new south wales             646
new york                    606
illinois                    438
istanbul                    416
western australia           363
queensland                  357
florida                     345
pennsylvania                344
nova scotia                 313
ohio                        304
virginia                    301
michigan                    300
indiana                     259
dubai                       256
gauteng                     239
georgia                     230
washington                  226
north carolina              223
massachusetts               219
new jersey                  218
south australia             

In [24]:
cities_raw = website_dataset.groupby('main_city')['root_domain'].count().sort_values(ascending = False)
print(cities_raw.head(60))
print(cities_raw.tail(60))

def clean_city(main_city):
    if not pd.isna(main_city):
        if not (main_city.strip()).isalpha():
            return main_city.strip().translate({ord(ch): None for ch in '0123456789'}).title().strip(' "').strip("'")
        elif main_city == ' Company':
            return np.nan
        else:
            return main_city.strip(" '").strip('"').title()

website_dataset['city_clean']=website_dataset['main_city'].apply(clean_city)

main_city
toronto           3293
calgary           2764
edmonton          1909
mississauga       1535
ottawa             978
montreal           929
london             890
winnipeg           854
quebec             649
brampton           629
saskatoon          578
markham            520
hamilton           502
vancouver          498
oakville           491
burlington         447
kitchener          380
richmond hill      373
barrie             371
regina             364
istanbul           362
scarborough        352
north york         344
windsor            318
woodbridge         304
dallas             279
concord            270
cambridge          257
etobicoke          250
dubai              250
guelph             250
red deer           237
lethbridge         234
waterloo           234
kingston           222
st. catharines     210
vaughan            201
grande prairie     197
milton             193
laval              180
newmarket          177
sherwood park      175
oshawa             170
b

In [ ]:
website_dataset.head(10)

,root_domain,domain_suffix,language,legal_name,main_city,main_country,main_region,phone,site_name,tld,s_category,site_name_clean,category_clean,legal_name_clean,country_clean,region_clean,city_clean
0,converterman.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1,clothesencounter.ca,ca,en,NaN,cardigan,canada,prince edward island,13066937766,Clothes Encounter,ca,Shoes & Other Footwear Stores,Clothes Encounter,Shoes & Other Footwear Stores,NaN,Canada,Prince Edward Island,Cardigan
2,investa.com.au,com.au,en,Investa Wholesale Funds Management Limited,brisbane,australia,queensland,61282269300,Investa Property Group,au,Real Estate Developers,Investa Property Group,Real Estate Developers,INVESTA WHOLESALE FUNDS MANAGEMENT LTD,Australia,Queensland,Brisbane
3,timminsgarage.com,com,en,Timmins Garage Inc.,timmins,canada,ontario,18775896640,Timmins Garage,com,Automobile Dealers & Manufacturers,Timmins Garage,Automobile Dealers & Manufacturers,TIMMINS GARAGE INC,Canada,Ontario,Timmins
4,destinate.ca,ca,en,Destinate Group Ltd.,NaN,Canada,NaN,NaN,Destinate Group,ca,Business Consulting,Destinate Group,Business Consulting,DESTINATE GROUP LTD,Canada,None,None
5,ironcrow.ca,ca,en,NaN,calgary,canada,alberta,14032878770,Iron Crow,ca,Furniture Stores,Iron Crow,Furniture Stores,NaN,Canada,Alberta,Calgary
6,springboarddm.com,com,en,The Hershey Company,mississauga,canada,ontario,19053690553,SpringBoard Data Management,com,Data Solutions,SpringBoard Data Management,Data Solutions,THE HERSHEY COMPANY,Canada,Ontario,Mississauga
7,stoneandtilesrus.com,com,en,NaN,brampton,canada,ontario,19054940660,STONE AND TILES R US,com,Tile Store,STONE AND TILES R US,Tile Store,NaN,Canada,Ontario,Brampton
8,mdaccpap.com,com,en,MDAC CPAP INC,thunder bay,canada,ontario,18076834405,MDAC CPAP,com,Medical Supply Manufacturers,MDAC CPAP,Medical Supply Manufacturers,MDAC CPAP INC,Canada,Ontario,Thunder Bay
9,micacchi.ca,ca,en,NaN,toronto,canada,ontario,16477257799,Micacchi Architecture,ca,Architects & Architectural Services,Micacchi Architecture,Architects & Architectural Services,NaN,Canada,Ontario,Toronto


In [25]:
website_dataset.columns

Index(['root_domain', 'domain_suffix', 'language', 'legal_name', 'main_city',
       'main_country', 'main_region', 'phone', 'site_name', 'tld',
       's_category', 'site_name_clean', 'category_clean', 'legal_name_clean',
       'country_clean', 'region_clean', 'city_clean'],
      dtype='object')

In [26]:
columns_to_keep = ['root_domain', 'domain_suffix', 'language', 'phone', 'site_name', 'tld',
                   'site_name_clean', 'category_clean', 'legal_name_clean',
       'country_clean', 'region_clean', 'city_clean']
website_dataset_clean = website_dataset[columns_to_keep].copy(deep = True)

del website_dataset, cities_raw, regions_raw, countries_raw, countries_cleaned, categories_raw, categories_cleaned
del site_names_raw, site_names_cleaned

In [27]:
with gcs.open('soleadify_sample_data/clean_data/website_dataset_clean.csv', 'w') as f:
    website_dataset_clean.to_csv(f,index=False)

After applying the cleaning on the website dataset, I will apply simillar cleaning to the Google and Facebook datasets. The purpose of this step is to create as much consistency as possible accross the 3 datasets before applying any sort of matching algorithm.

In [28]:
google_dataset.describe(include='all')

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
count,330552,304870,311002,304084,311039,356489,3.239480e+05,241258,311875,328160,310849,310854,353005,273400,356520
unique,282513,470,22200,213,187,353898,NaN,210,140744,293157,678,1508,349054,103062,72010
top,"Montreal, Quebec, Canada",Clinics - Surgeons & Physicians,calgary,us,united states,Lincoln Elementary School,NaN,ca,"Toronto, ON, Canada",+44 345 722 3344,on,ontario,"No reviews · University Guildford, United King...",97239,facebook.com
freq,1111,17229,4591,110083,132132,21,NaN,90520,1593,748,40046,40045,48,667,71983
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.335193e+11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,2.853369e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,4.350213e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,1.425494e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,1.708520e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,3.318566e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
domain_count = google_dataset.groupby('domain').size().sort_values(ascending = False)
print(domain_count.head(60))
print(domain_count.tail(60))

domain
facebook.com                  71983
postoffice.co.uk               6010
instagram.com                  5647
ihg.com                        4356
marriott.com                   3979
hilton.com                     3477
doctolib.fr                    3105
ameripriseadvisors.com         2378
choicehotels.com               1625
uhaul.com                      1518
gov.uk                         1511
bit.ly                         1486
spar.co.uk                     1428
411habitation.com              1097
bestwestern.com                1093
etsy.com                       1054
dignitymemorial.com            1050
uchealth.org                   1032
anchorhanover.org.uk           1012
vagaro.com                      992
iuhealth.org                    991
ochsner.org                     978
sanfordhealth.org               959
motel6.com                      929
kindercare.com                  877
homehardware.ca                 857
uwmedicine.org                  851
wexnermedical.osu.edu

In [ ]:
google_dataset.query('domain == "youtube.com"').head(5)

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain
900,NaN,NaN,NaN,us,NaN,Skyway ENT LLC,1.727679e+10,us,NaN,+1 727-678-5514,NaN,NaN,No reviews · Record company Open 24 hours · +1...,NaN,youtube.com
901,NaN,NaN,NaN,NaN,NaN,Cesar Sanchez,1.704500e+10,NaN,NaN,(704) 500-3789,NaN,NaN,5.0 (1) · Musical club (704) 500-3789,NaN,youtube.com
902,NaN,NaN,NaN,NaN,NaN,Hourglass Nature Films,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,youtube.com
903,NaN,"Musicians, Orchestras & Bands",NaN,NaN,NaN,Vegaz Valentine,1.414999e+10,NaN,NaN,(414) 998-8242,NaN,NaN,No reviews · Musician (414) 998-8242,NaN,youtube.com
904,COMPANY IMK PRODUCTION، Rue du Midi 51، 1000 B...,"Alarms, Surveillance & Security Systems",bruxelles,be,belgium,COMPANY IMK PRODUCTION,3.245642e+10,be,"Brussels, Belgium · Near the Manneken Pis",+32 456 41 58 92,bru,bruxelles-capitale,"No reviews · Cable company Brussels, Belgium ·...",1000,youtube.com


Based on the output above it seems that the domain is not a very reliable way of uniquely identifying a company. There are a lot of them which have facebook.com, instagram.com, youtube.com as their domain. They're probably Facebook pages or Instagram / Youtube channels, but for the purpose of building a better dataset, it's not useful for linking data.

In [29]:
countries_raw = google_dataset.groupby('country_name').size().sort_values(ascending = False)
print(countries_raw.head(30))
print(countries_raw.tail(30))
google_dataset['country_name_consistent'] = google_dataset['country_name'].apply(lambda x: x.strip().title() if not pd.isna(x) else x)
del countries_raw

country_name
united states           132132
canada                   86447
united kingdom           42311
australia                12612
france                    8960
germany                   2401
new zealand               2239
malaysia                  2036
belgium                   2006
south africa              1733
india                     1555
turkey                    1474
united arab emirates      1437
netherlands               1263
russia                    1183
japan                      954
mexico                     817
spain                      564
philippines                535
italy                      463
hong kong                  394
poland                     388
taiwan                     351
qatar                      325
ireland                    260
vietnam                    250
greece                     247
china                      234
switzerland                234
denmark                    219
dtype: int64
country_name
seychelles                     

In [30]:
country_codes_raw = google_dataset.groupby('country_code').size().sort_values(ascending = False)
print(country_codes_raw.head(20))
print(country_codes_raw.tail(20))

def clean_codes(country_code):
    if not pd.isna(country_code):
        if country_code == '001':
            return np.nan
        else:
            return country_code.strip().upper()
    else:
        return np.nan

google_dataset['country_code_consistent'] = google_dataset['country_code'].apply(clean_codes)
google_dataset['phone_country_code_consistent'] = google_dataset['phone_country_code'].apply(clean_codes)

country_code
us    110083
ca     95379
gb     44919
au     12637
fr      9432
nz      2429
de      2314
be      2030
my      1757
za      1681
tr      1643
in      1480
ae      1394
nl      1229
jp      1078
ru       994
mx       959
sg       797
hk       664
ph       549
dtype: int64
country_code
001    2
td     2
cf     2
ss     2
mh     2
tm     1
pf     1
gd     1
gi     1
fo     1
dm     1
ls     1
vg     1
sl     1
nr     1
ci     1
kp     1
gw     1
km     1
im     1
dtype: int64


There are 3 fields which point to the same information - country_code, country_name, phone_country_code. These all represent a form of Country information. I will unify them in a single field.

In [ ]:
print('''There are {} instances where the country code does not exist, 
but the phone country code exists'''
.format(google_dataset[google_dataset['country_code_consistent'].isnull() & google_dataset['phone_country_code_consistent'].notna()].shape[0]))

There are 0 instances where the country code does not exist, 
but the phone country code exists


This means that the phone country code is redundant at this step. I can turn it into a dialing code at a later point using country information.

In [32]:
with gcs.open(pathFiles+'iso_codes.csv') as f:
    countries_iso = pd.read_csv(f)
countries_iso.head()

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS


In [33]:
print(countries_iso.query('Name == "Namibia"'))
print(countries_iso[countries_iso['Code'].isna()])
countries_iso['Code'] = countries_iso['Code'].fillna('NA')
print(countries_iso.query('Name == "Namibia"'))

        Name Code
153  Namibia  NaN
        Name Code
153  Namibia  NaN
        Name Code
153  Namibia   NA


In [34]:
google_dataset = google_dataset.merge(countries_iso, how = 'left', left_on = 'country_code_consistent', right_on = 'Code')


In [ ]:
google_dataset.query('country_code_consistent == "NA"').head(5)

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain,country_name_consistent,country_code_consistent,phone_country_code_consistent,Name,Code
11045,"Nelson Mandela Ave, Windhoek, Namibia",Breweries,windhoek,na,namibia,Embassy Liquor Store,2.646124e+10,na,"Windhoek, Namibia",+264 61 237 028,kh,khomas,"4.7 (45) · Liquor wholesaler Windhoek, Namibia...",NaN,facebook.com,Namibia,NA,NA,Namibia,NA
53124,NaN,Airline Companies,NaN,na,NaN,South African Airways Office,2.646127e+10,na,NaN,+264 61 273 350,NaN,NaN,3.8 (20) · Airline Namibia Closed ⋅ Opens 8AM ...,NaN,flysaa.com,NaN,NA,NA,Namibia,NA
83134,"116 Robert Mugabe Ave, Windhoek, Namibia","Executive, Legislative & Government Institutions",windhoek,na,namibia,British High Commission Windhoek,2.646127e+10,na,"Windhoek, Namibia",+264 61 274 800,kh,khomas,"5.0 (4) · Government office Windhoek, Namibia ...",NaN,gov.uk,Namibia,NA,NA,Namibia,NA
83724,"B1 Main Road, Ondangwa, Namibia",NaN,ondangwa,na,namibia,Cashbuild,2.646524e+10,na,NaN,+264 65 240 501,on,oshana,"Ondangwa, Namibia · +264 65 240 501 Closed ⋅ O...",NaN,cashbuild.co.za,Namibia,NA,NA,Namibia,NA
88008,"Hendrik Witbooi, Swakopmund, Namibia",Industrial Machinery & Supplies,swakopmund,na,namibia,Sasol Namibia,2.646441e+10,na,"Swakopmund, Namibia",+264 64 406 222,er,erongo,"5.0 (1) · Corporate office Swakopmund, Namibia...",NaN,sasol.com,Namibia,NA,NA,Namibia,NA


In [ ]:
google_dataset[google_dataset['Code'].isna()].head(5)

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,region_code,region_name,text,zip_code,domain,country_name_consistent,country_code_consistent,phone_country_code_consistent,Name,Code
4,NaN,Catering & Delivery,NaN,NaN,NaN,Lakay express,NaN,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee,NaN,NaN,NaN,NaN,NaN
13,"1800 Orleans St, Baltimore, MD 21287",Clinics - Surgeons & Physicians,baltimore,NaN,united states,The Johns Hopkins Hospital Emergency Room,1.410956e+10,NaN,"Baltimore, MD · (410) 955-5000",(410) 955-5000,md,maryland,"Emergency room Baltimore, MD · (410) 955-5000 ...",21287,hopkinsmedicine.org,United States,NaN,NaN,NaN,NaN
17,"363 Horton St E, London, ON N6B 1L6, Canada",Beauty Salons - Equipment & Supplies,london,NaN,canada,Halo Hair Co,NaN,NaN,363 Horton St E,NaN,on,ontario,4.7 (3) · Hairdresser 363 Horton St E Closed ⋅...,n6b 1l6,instagram.com,Canada,NaN,NaN,NaN,NaN
21,"Europa House, Newton Ln, Wigston LE18 3SE, Uni...",Beauty Salons,south wigston,NaN,united kingdom,Claws Nail Studio,NaN,NaN,NaN,NaN,eng,england,No reviews · Nail salon United Kingdom Tempora...,le18 3se,instagram.com,United Kingdom,NaN,NaN,NaN,NaN
28,NaN,NaN,NaN,NaN,NaN,Daniel Thomas Weddings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,instagram.com,NaN,NaN,NaN,NaN,NaN


In [35]:
country_analysis = google_dataset.groupby(['country_name_consistent','Name']).size()
print(country_analysis.head(40))

country_name_consistent  Name                     
Afghanistan              Afghanistan                      4
Albania                  Albania                         32
Algeria                  Algeria                         20
                         Ukraine                          1
American Samoa           American Samoa                   2
Andorra                  Andorra                         12
Angola                   Angola                           8
Argentina                Argentina                      113
                         Belgium                          1
                         Nigeria                          1
                         South Africa                     1
Armenia                  Armenia                         16
Aruba                    Aruba                            8
Australia                Australia                    11830
                         France                           1
                         Kenya                   

In [ ]:
print('''There are {0:,} instances where the country name is not originally present,
but based on the code the country can be determined'''
.format(google_dataset[google_dataset['country_name_consistent'].isna() & google_dataset['Name'].notna()].shape[0]))

There are 26,651 instances where the country name is not originally present,
but based on the code the country can be determined


In [36]:
def fill_country(row):
    if not pd.isna(row['country_name_consistent']):
        return row['country_name_consistent']
    else:
        return row['Name']
google_dataset['country_final']=google_dataset[['country_name_consistent','Name']].apply(lambda row: fill_country(row),axis=1)

google_dataset[['country_name_consistent','country_final']].describe()

,country_name_consistent,country_final
count,311039,337690
unique,187,234
top,United States,United States
freq,132132,136976


In [37]:
cities_raw = google_dataset.groupby('city').size().sort_values(ascending = False)
print(cities_raw.head(60))
print(cities_raw.tail(60))

city
calgary             4591
toronto             3769
edmonton            3428
london              3374
montreal            2780
mississauga         2729
new york            1782
quebec              1766
ottawa              1705
winnipeg            1625
north york          1607
indianapolis        1562
dallas              1536
brampton            1407
richmond            1394
bristol             1350
columbus            1330
charlotte           1330
seattle             1326
houston             1323
vancouver           1294
portland            1240
oklahoma city       1239
hamilton            1217
philadelphia        1214
boston              1164
madison             1115
scarborough         1112
saskatoon           1103
phoenix             1051
cambridge           1010
burlington          1007
markham              949
atlanta              944
chicago              925
birmingham           915
aurora               914
baltimore            912
laval                889
windsor             

In [ ]:
cities_raw[cities_raw.index.str.startswith('moskva')].head(20)

city
moskva 90     1
moskva 404    1
moskva 342    1
dtype: int64

In [ ]:
cities_raw[cities_raw.index.str.startswith('dublin')].head(20)

city
dublin       196
dublin 22      3
dublin 2       2
dublin 1       2
dublin 7       1
dublin 4       1
dublin 24      1
dublin 15      1
dtype: int64

In [38]:
regions_raw = google_dataset.groupby('region_name').size().sort_values(ascending = False)
print(regions_raw.head(60))
print(regions_raw.tail(60))

region_name
ontario                 40045
england                 37009
quebec                  16999
alberta                 13591
texas                   11306
california              10293
british columbia         6552
florida                  5206
new york                 4846
north carolina           4454
ohio                     4366
indiana                  4179
virginia                 3817
arizona                  3809
washington               3775
pennsylvania             3759
new south wales          3672
michigan                 3661
massachusetts            3585
wisconsin                3513
illinois                 3439
colorado                 3418
saskatchewan             3098
georgia                  3072
louisiana                3070
victoria                 3069
oklahoma                 2826
kentucky                 2716
scotland                 2694
tennessee                2693
queensland               2435
manitoba                 2390
oregon                   235

In [ ]:
regions_raw[regions_raw.index.str.startswith('dublin')]

region_name
dublin 22    3
dublin 1     2
dublin 15    1
dublin 2     1
dublin 24    1
dublin 4     1
dublin 7     1
dtype: int64

In [ ]:
regions_raw[regions_raw.index.str.startswith('moskva')]

region_name
moskva    3
dtype: int64

In [40]:
def clean_region_or_city(region_or_city):
    if not pd.isna(region_or_city):
        if not (region_or_city.strip()).isalpha():
            return region_or_city.strip().translate({ord(ch): None for ch in '0123456789'}).title().strip(' "').strip("'")
        else:
            return region_or_city.strip(" '").strip('"').title()
    
    else:
        return np.nan

google_dataset['city_consistent'] = google_dataset['city'].apply(clean_region_or_city)
google_dataset['region_consistent'] = google_dataset['region_name'].apply(clean_region_or_city)

In [41]:
google_dataset[google_dataset['city'].str.startswith('moskva') & google_dataset['city'].notna()].head()

,address,category,city,country_code,country_name,name,phone,phone_country_code,raw_address,raw_phone,...,zip_code,domain,country_name_consistent,country_code_consistent,phone_country_code_consistent,Name,Code,country_final,city_consistent,region_consistent
225888,"140125, Московская обл, 1-я Стекольная ул., 1,...",Glass & Mirror Stores,moskva 404,NaN,russia,Pilkington Glass,7.495370e+10,NaN,"140125, Московская обл, 1-я Стекольная ул., 1",(495) 369-95-00,...,115404,firstinglass.ru,Russia,NaN,NaN,NaN,NaN,Russia,Moskva,Moskva
270379,"Миклухо-Маклая ул 42 Москва, АО, Russia, 117342",NaN,moskva 342,ru,russia,Moltomolto,7.495545e+10,ru,"АО, Moscow, Russia",+7 495 545-48-89,...,117342,moltomolto.ru,Russia,RU,RU,Russian Federation,RU,Russia,Moskva,Moskva
320679,"129090, Москва, Троицкая ул., д, Troitskaya Ul...",Energy - Equipment & Supplies,moskva 90,ru,russia,Corporation GazEnergoStroy,7.495792e+10,ru,"Moscow, Russia",+7 495 792-39-42,...,129090,gazenergostroy.ru,Russia,RU,RU,Russian Federation,RU,Russia,Moskva,Moskva


In [ ]:
google_dataset.columns

Index(['address', 'category', 'city', 'country_code', 'country_name', 'name',
       'phone', 'phone_country_code', 'raw_address', 'raw_phone',
       'region_code', 'region_name', 'text', 'zip_code', 'domain',
       'country_name_consistent', 'country_code_consistent',
       'phone_country_code_consistent', 'Name', 'Code', 'country_final',
       'city_consitent', 'region_consitent'],
      dtype='object')

In [42]:
columns_to_keep = ['address', 'category', 'name',
       'phone', 'raw_address', 'raw_phone',
       'region_code', 'region_name', 'text', 'zip_code', 'domain',
        'country_final',
       'city_consistent', 'region_consistent']
google_dataset_clean = google_dataset[columns_to_keep]
google_dataset_clean.head()

,address,category,name,phone,raw_address,raw_phone,region_code,region_name,text,zip_code,domain,country_final,city_consistent,region_consistent
0,"28 Central Coast Hwy, West Gosford NSW 2250, A...",Fabric-Based Home Goods,Spotlight West Gosford,6.124336e+10,"West Gosford NSW, Australia",+61 2 4335 5946,nsw,new south wales,"4.1 (766) · Craft store West Gosford NSW, Aust...",2250,spotlightstores.com,Australia,Gosford,New South Wales
1,"400 Scott St, St. Catharines, ON L2M 3W2, Canada",Book Stores,Heritage Christian Book Store,1.905937e+10,400 Scott St · In Grantham Plaza,+1 905-937-4553,on,ontario,4.7 (100) · Book store 400 Scott St · In Grant...,l2m 3w2,bookmanager.com,Canada,St. Catharines,Ontario
2,"191 Pleasant St, Yarmouth, NS B5A 2J9, Canada",Other Building Material Retailers,Pleasant Timber Mart,1.902743e+10,"7+ years in business · Yarmouth, NS, Canada",+1 902-742-9181,ns,nova scotia,4.7 (40) · Building materials store 7+ years i...,b5a 2j9,timbermart.ca,Canada,Yarmouth,Nova Scotia
3,"11040 Santa Monica Blvd Suite 370, Los Angeles...",Plastic Surgery Clinics,Skin Specifics Medical Spa West LA,1.818427e+10,7+ years in business · 11040 Santa Monica Blvd...,+1 818-426-8353,ca,california,4.3 (15) · Medical spa 7+ years in business · ...,90025,linktr.ee,United States,Los Angeles,California
4,NaN,Catering & Delivery,Lakay express,NaN,NaN,NaN,NaN,NaN,5.0 (1) · Mobile caterer Open ⋅ Closes 9PM,NaN,linktr.ee,NaN,NaN,NaN


In [43]:
with gcs.open('soleadify_sample_data/clean_data/google_dataset_clean.csv', 'w') as f:
    google_dataset_clean.to_csv(f,index=False, quotechar='"', escapechar='\\')

In [44]:
facebook_dataset.describe(include='all')

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code
count,72010,57380,55310,45106,57874,45404,28003,20289,72010,72009,72004,4.486600e+04,38013,45101,45101,36049
unique,72010,57121,10184,6353,130,117,27625,20280,72010,71837,4,NaN,100,435,686,26223
top,euro-hygiene-34.fr,"calgary, ab, canada, t2g, calgary, ab, canada,...",Auto Services,toronto,ca,canada,History Museum,ford.can.gp@gmail.com,https://euro-hygiene-34.fr,Nissan,LocalBusiness,NaN,ca,on,ontario,victoria
freq,1,9,935,2713,41955,31160,273,7,1,7,52747,NaN,33726,16710,16710,463
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.517792e+10,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.945566e+12,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.114030e+06,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.416606e+10,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.604368e+10,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.815358e+10,NaN,NaN,NaN,NaN


In [ ]:
# To do - make country, region, city consitent using a similar approach to google dataset
# Again country code seems to have more coverage than country. To be combined with phone country code and then with iso codes
# domain seems to be unique
# phone is different than from google
# The best approach would be to combine the 3 datasets using common elements - Country, City/Region
# Then combine and keep only 1 using string matchin

In [8]:
countries_raw = facebook_dataset.groupby('country_name').size().sort_values(ascending = False)
print(countries_raw.head(60))
print(countries_raw.tail(60))

country_name
canada                  31160
united states            6367
australia                1614
united kingdom            939
russia                    508
united arab emirates      424
south africa              419
germany                   391
turkey                    385
netherlands               376
belgium                   317
malaysia                  294
france                    282
singapore                 222
india                     164
hong kong                 157
greece                    146
taiwan                    145
poland                    124
spain                     103
south korea                71
philippines                59
china                      55
new zealand                55
italy                      47
vietnam                    47
japan                      42
austria                    31
sweden                     28
bangladesh                 24
indonesia                  23
switzerland                20
norway                     

In [9]:
country_codes_raw = facebook_dataset.groupby('country_code').size().sort_values(ascending = False)
print(country_codes_raw.head(60))
print(country_codes_raw.tail(60))

country_code
ca    41955
us     6703
au     1670
gb     1240
ru      538
tr      508
ae      456
za      445
de      441
nl      393
jp      336
be      331
my      313
fr      294
sg      227
in      212
gr      173
hk      163
tw      162
pl      140
kr      125
es      115
nz       72
ph       64
cn       57
vn       55
it       52
bd       35
at       32
qa       30
se       29
id       28
lk       22
mx       21
ch       20
no       19
dk       19
th       16
br       16
ec       15
co       15
ro       14
pa       13
sa       12
fi       12
om       12
ar       12
pk       11
cl       10
ie        9
eg        9
hu        8
gt        7
ua        7
cz        7
il        7
sv        7
pr        7
kw        6
ee        6
dtype: int64
country_code
ve    4
bs    3
lt    3
rs    3
bg    2
by    2
ba    2
bb    2
gi    2
ge    2
bz    2
zw    2
iq    2
lu    2
is    2
sk    2
mz    2
mt    2
md    2
py    2
ug    2
tg    2
vg    2
zm    2
vi    2
uy    1
sn    1
si    1
xk    1
bw    1
a

In [10]:
facebook_dataset[facebook_dataset['country_name'].isna() & facebook_dataset['country_code'].notna()].shape[0]

12471

In [11]:
facebook_dataset[facebook_dataset['country_code'].isna() & facebook_dataset['phone_country_code'].notna()].shape[0]

0

In [12]:
region_names_raw = facebook_dataset.groupby('region_name').size().sort_values(ascending = False)
print(region_names_raw.head(60))
print(region_names_raw.tail(60))

region_name
ontario              16710
alberta               6487
quebec                3764
british columbia      1541
saskatchewan          1217
manitoba              1036
england                896
texas                  888
victoria               482
california             471
new south wales        470
new york               343
illinois               300
pennsylvania           294
dubai                  283
virginia               262
western australia      259
moscow                 256
gauteng                250
indiana                229
michigan               220
queensland             218
ohio                   213
istanbul               198
nova scotia            196
florida                188
flanders               178
wisconsin              161
massachusetts          160
washington             153
north carolina         148
new jersey             141
connecticut            139
south australia        139
georgia                138
new brunswick          115
minnesota       

In [13]:
cities_raw = facebook_dataset.groupby('city').size().sort_values(ascending = False)
print(cities_raw.head(60))
print(cities_raw.tail(60))

city
toronto            2713
calgary            2529
edmonton           1792
mississauga        1580
ottawa              816
winnipeg            799
brampton            653
london              642
markham             553
saskatoon           529
montreal            508
oakville            497
hamilton            482
burlington          441
richmond hill       403
quebec              401
kitchener           383
vancouver           369
dallas              361
regina              356
barrie              351
windsor             338
woodbridge          325
dubai               283
guelph              260
moscow              260
cambridge           246
lethbridge          239
red deer            230
waterloo            226
grande prairie      204
kingston            202
newmarket           191
vaughan             188
oshawa              179
sherwood park       173
milton              171
whitby              169
brantford           161
greater sudbury     156
aurora              155
peterboroug

In [45]:
facebook_dataset['country_name_consistent'] = facebook_dataset['country_name'].apply(lambda x: x.strip().title() if not pd.isna(x) else x)

In [46]:
def clean_codes(country_code):
    if not pd.isna(country_code):
        if country_code == '001':
            return np.nan
        else:
            return country_code.strip().upper()
    else:
        return np.nan

facebook_dataset['country_code_consistent'] = facebook_dataset['country_code'].apply(clean_codes)

In [47]:
with gcs.open(pathFiles + 'iso_codes.csv') as f:
    countries_iso = pd.read_csv(f, header = 0, names = ['ISO_Country_Name', 'ISO_2_Country_Code'])
    
countries_iso['ISO_2_Country_Code'] = countries_iso['ISO_2_Country_Code'].fillna('NA')

facebook_dataset = facebook_dataset.merge(countries_iso, how = 'left', left_on = 'country_code_consistent', right_on = 'ISO_2_Country_Code')

def fill_country(row):
    if not pd.isna(row['country_name_consistent']):
        return row['country_name_consistent']
    else:
        return row['ISO_Country_Name']

facebook_dataset['country_final']=facebook_dataset[['country_name_consistent','ISO_Country_Name']].apply(lambda row: fill_country(row),axis=1)

facebook_dataset[['country_name_consistent','country_final']].describe()

,country_name_consistent,country_final
count,45404,57875
unique,117,142
top,Canada,Canada
freq,31160,41955


In [17]:
"leningradskaia oblast'".isalpha()

False

In [48]:
def clean_region_or_city(region_or_city):
    if not pd.isna(region_or_city):
        if not (region_or_city.strip()).isalpha():
            return region_or_city.strip().translate({ord(ch): None for ch in '0123456789'}).title().strip(" '").strip('"')
        elif region_or_city in ["leningradskaia oblast'","leningradskaya oblast'"]:
            return "Leningradskaya Oblast"
        elif region_or_city == "st.-petersburg":
            return "St Petersburg"
        else:
            return region_or_city.strip(" '").strip('"').title()
    else:
        return np.nan

facebook_dataset['city_consistent'] = facebook_dataset['city'].apply(clean_region_or_city)
facebook_dataset['region_consistent'] = facebook_dataset['region_name'].apply(clean_region_or_city)

In [19]:
facebook_dataset.head()

,domain,address,categories,city,country_code,country_name,description,email,link,name,page_type,phone,phone_country_code,region_code,region_name,zip_code,country_name_consistent,country_code_consistent,ISO_Country_Name,ISO_2_Country_Code,country_final,city_consistent,region_consistent
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,calvisson,fr,france,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,NaN,occ,occitanie,30420,France,FR,France,FR,France,Calvisson,Occitanie
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,ca,NaN,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,ca,NaN,NaN,NaN,NaN,CA,Canada,CA,Canada,None,None
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,NaN,NaN,NaN,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,vancouver,ca,canada,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,ca,bc,british columbia,v6b 1g1,Canada,CA,Canada,CA,Canada,Vancouver,British Columbia
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,winnipeg,ca,canada,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,ca,mb,manitoba,r3l 2t7,Canada,CA,Canada,CA,Canada,Winnipeg,Manitoba


In [49]:
facebook_dataset.columns

Index(['domain', 'address', 'categories', 'city', 'country_code',
       'country_name', 'description', 'email', 'link', 'name', 'page_type',
       'phone', 'phone_country_code', 'region_code', 'region_name', 'zip_code',
       'country_name_consistent', 'country_code_consistent',
       'ISO_Country_Name', 'ISO_2_Country_Code', 'country_final',
       'city_consistent', 'region_consistent'],
      dtype='object')

In [50]:
columns_to_keep = ['domain', 'address', 'categories', 
 'description', 'email', 'link', 'name', 'page_type',
       'phone',  'region_code',  'zip_code', 'country_final',
       'city_consistent', 'region_consistent']
facebook_dataset_clean = facebook_dataset[columns_to_keep]
facebook_dataset_clean.head()

,domain,address,categories,description,email,link,name,page_type,phone,region_code,zip_code,country_final,city_consistent,region_consistent
0,euro-hygiene-34.fr,"134 rue entrepreneurs, za du vigné, 30420, cal...",NaN,NaN,NaN,https://euro-hygiene-34.fr,Euro Hygiène,LocalBusiness,NaN,occ,30420,France,Calvisson,Occitanie
1,lakesidehomeservices.ca,NaN,Appliance Repair & Maintenance|Home Builders &...,NaN,NaN,https://lakesidehomeservices.ca,Lakeside Home Services,LocalBusiness,1.613832e+10,NaN,NaN,Canada,NaN,NaN
2,rossiterboats.com,NaN,Boats & Yachts Dealers|Boat Tours & Cruises,At Rossiter Boats we build semi-custom power b...,info@rossiterboats.com,http://rossiterboats.com,Rossiter Boats,LocalBusiness,NaN,NaN,NaN,NaN,NaN,NaN
3,caa-architecture.com,"601 cordova st w # 270, v6b 1g1, vancouver, bc...",Architects & Architectural Services|Other Engi...,NaN,office@caa-arcitecture.com,http://caa-architecture.com,Chandler Associates Architecture Inc.,LocalBusiness,1.604687e+10,bc,v6b 1g1,Canada,Vancouver,British Columbia
4,apexsurety.ca,"unit 3 - 4 donald street, r3l 2t7, winnipeg, m...",NaN,NaN,NaN,https://apexsurety.ca,Apex Surety & Insurance Ltd.,LocalBusiness,1.204560e+10,mb,r3l 2t7,Canada,Winnipeg,Manitoba


In [51]:
with gcs.open('soleadify_sample_data/clean_data/facebook_dataset_clean.csv','w') as f:
    facebook_dataset_clean.to_csv(f, index = False  , quotechar='"', escapechar='\\')